In [456]:
import numpy as np
import torch
import rms_torch

H = np.load("array/torch/BLBQ1D_loc/J0_3_J1_1_hx_0_hz_0/1_mel/H/0.npy")
# H2 = np.load("array/torch/HXYZ1D_loc/Jx_0.04851_Jy_5_Jz_1.043_hx_0.4123_hz_0_lt_1/H/0.npy")
u = np.load("/Users/keisukemurota/Documents/todo/worms/python/rmsKit/array/torch/BLBQ1D_loc/J0_3_J1_1_hx_0_hz_0/1_mel/Adam/lr_0.01_epoch_2000/loss_0.00000/u/0.npy")


In [457]:
h_list = [-H]
loss = rms_torch.SystemQUasiEnergyLoss(h_list, device="cpu")
model = rms_torch.UnitaryRieman(
    h_list[0].shape[1], 3, device="cpu").to("cpu")
model.reset_params(torch.from_numpy(u))
loss(model())

tensor(-5.7049e-08, dtype=torch.float64, grad_fn=<SubBackward0>)

In [458]:
np.round(h_list[0],5)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -3.,  0.,  3.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -5.,  0.,  2.,  0.,  1.,  0.,  0.],
       [ 0.,  3.,  0., -3.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  2.,  0., -2.,  0.,  2.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -3.,  0.,  3.,  0.],
       [ 0.,  0.,  1.,  0.,  2.,  0., -5.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  3.,  0., -3.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [459]:
u

array([[-7.07106709e-01, -6.15873786e-08, -7.07106854e-01],
       [-2.06606316e-02, -9.99573045e-01,  2.06607144e-02],
       [-7.06804953e-01,  2.92186039e-02,  7.06804805e-01]])

In [460]:
from utils import stoquastic
import utils

In [466]:
u0 = np.array([
    [1/np.sqrt(2),0,-1/np.sqrt(2)],
    [1/np.sqrt(2),0,1/np.sqrt(2)],
    [0,1,0]
])
U = np.kron(u,u)

In [467]:
L = 6
h1 = - H
h2 = stoquastic(-U @ H @ U.T)
HS1 = utils.sum_ham(h1, [[i, (i+1)%L] for i in range(L)], L, 3)
HS2 = utils.sum_ham(h2, [[i, (i+1)%L] for i in range(L)], L, 3)

In [468]:
e2, v2 = np.linalg.eigh(h2)
e1, v1 = np.linalg.eigh(h1)

In [469]:
e1

array([-2.00000002, -2.        , -2.        , -1.99999997,  3.99999983,
        3.99999983,  4.        ,  4.        ,  4.00000006])

In [470]:
e2

array([-2.00000118, -2.        , -1.99999997, -1.99999884,  3.99999983,
        3.99999983,  4.        ,  4.        ,  4.00000006])

In [445]:
J = np.array([-1, 5, 0.3])
h = np.array([0.1, 0, 0.4])

In [313]:
import numpy as np
from lattice.core.constants import *

In [314]:
SySy

array([[ 0.  ,  0.  ,  0.  , -0.25],
       [ 0.  ,  0.  ,  0.25,  0.  ],
       [ 0.  ,  0.25,  0.  ,  0.  ],
       [-0.25,  0.  ,  0.  ,  0.  ]])

In [315]:
SzSz

array([[ 0.25,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.25,  0.  , -0.  ],
       [ 0.  ,  0.  , -0.25, -0.  ],
       [ 0.  , -0.  , -0.  ,  0.25]])

In [87]:
import numpy as np

def cartesian_to_spherical(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z / r)
    phi = np.arctan2(y, x)
    return r, theta, phi

def rotation_matrix(theta, phi):
    # Convert angles from degrees to radians

    # Rotation matrix around the x-axis
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(phi), -np.sin(phi)],
                   [0, np.sin(phi), np.cos(phi)]])
    
    
    # Rotation matrix around the z-axis
    Rz = np.array([[np.cos(theta), 0, -np.sin(theta)],
                   [0, 1, 0],
                   [np.sin(theta), 0, np.cos(theta)]])
    
    print(Rz)

    # Combined rotation matrix (Rz first, then Rx)
    R = np.dot(Rz, Rx)

    return R

r, theta, phi = cartesian_to_spherical(h[0], h[1], h[2])

In [88]:
theta

0.2449786631268647

In [97]:
theta

0.2449786631268647

In [93]:
import scipy
U2 = scipy.linalg.expm(-1j * theta * Sy)

In [99]:
U2.T.conj() @ Sx @ U2

array([[ 0.12126782+0.j,  0.48507127+0.j],
       [ 0.48507127+0.j, -0.12126782+0.j]])

In [105]:
J = np.array([-1, 5, 0.3])
h = np.array([0, 0, 0])
rot = (np.pi/2 - theta)/2
R = rotation_matrix(0.1, phi)


h = R.T @ R.T @ h
J = R.T @ J
h_single = h[2] * Sz + h[0] * Sx
h_single = np.kron(h_single, I2) + np.kron(I2, h_single)

H = J[0] * SxSx + J[1] * SySy + J[2] * SzSz + h_single
np.linalg.eigvalsh(H)

[[ 0.99500417  0.         -0.09983342]
 [ 0.          1.          0.        ]
 [ 0.09983342  0.          0.99500417]]


array([-1.39167987, -1.10832013,  0.9091528 ,  1.5908472 ])

In [32]:
H

array([[-4.00000000e+00, -0.00000000e+00, -1.71142705e-08,
        -0.00000000e+00, -6.84570831e-08, -0.00000000e+00,
        -1.71142705e-08, -0.00000000e+00, -2.92898264e-16],
       [-0.00000000e+00, -9.99999966e-01, -0.00000000e+00,
        -2.99999995e+00, -0.00000000e+00, -5.13428123e-08,
        -0.00000000e+00, -3.42285410e-08, -0.00000000e+00],
       [-1.71142705e-08, -0.00000000e+00,  1.00000003e+00,
        -0.00000000e+00, -1.99999997e+00, -0.00000000e+00,
        -9.99999966e-01, -0.00000000e+00, -1.71142705e-08],
       [-0.00000000e+00, -2.99999995e+00, -0.00000000e+00,
        -9.99999966e-01, -0.00000000e+00, -3.42285410e-08,
        -0.00000000e+00, -5.13428123e-08, -0.00000000e+00],
       [-6.84570831e-08, -0.00000000e+00, -1.99999997e+00,
        -0.00000000e+00, -1.99999993e+00, -0.00000000e+00,
        -1.99999997e+00, -0.00000000e+00, -6.84570831e-08],
       [-0.00000000e+00, -5.13428123e-08, -0.00000000e+00,
        -3.42285410e-08, -0.00000000e+00, -9.999999